In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE

In [35]:
!pip install imblearn

     -------------------------------------- 226.0/226.0 kB 3.4 MB/s eta 0:00:00
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0


In [19]:
concatenated_matches_infos_dataframe = pd.read_pickle("../machine_learning/dataframes/data")

percentage_of_win_per_team_after_won_game_1 = concatenated_matches_infos_dataframe[concatenated_matches_infos_dataframe["winner_id"] == concatenated_matches_infos_dataframe["Game 1 winner_id"]].groupby(
    "team_id").size() / concatenated_matches_infos_dataframe.groupby("team_id").size() * 100

percentage_of_win_per_team_after_won_game_1.name = "percentage_of_win_per_team_after_won_game_1"

concatenated_matches_infos_dataframe = pd.merge(concatenated_matches_infos_dataframe, percentage_of_win_per_team_after_won_game_1.to_frame(), left_on="team_id", right_index=True, how="left")

concatenated_matches_infos_dataframe[(concatenated_matches_infos_dataframe["team_id"] == 100) & (concatenated_matches_infos_dataframe["Game 1 winner_id"] == 100)].columns

,match_id,name,tournament_id,number_of_games,winner_id,team_id,opponent_id,tournament_tier,tournament_has_bracket,tournament_name,Game 1 winner_id,Game 2 winner_id,Game 3 winner_id,Game 4 winner_id,Game 5 winner_id,percentage_of_match_win_after_game_1
0,524964,DW vs MMM,2038,1,2796,180,2796,None,0,Regular season,2796,-1,-1,-1,-1,NaN
1,524965,MMM vs LGC,2038,1,2796,2796,181,None,0,Regular season,2796,-1,-1,-1,-1,NaN
2,524968,BMR vs AV,2038,1,1519,1519,179,None,0,Regular season,1519,-1,-1,-1,-1,NaN
3,524970,MMM vs BMR,2038,1,1519,2796,1519,None,0,Regular season,1519,-1,-1,-1,-1,NaN
4,524971,ORD vs DW,2038,1,1521,180,1521,None,0,Regular season,1521,-1,-1,-1,-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43969,770192,BLX vs JL,10394,1,132476,132476,132656,d,0,Regular Season,132476,-1,-1,-1,-1,NaN
43970,770193,WYL vs SOL,10394,1,130452,133070,130452,d,0,Regular Season,130452,-1,-1,-1,-1,NaN
43971,770194,AKR vs GM,10394,1,133115,133115,133071,d,0,Regular Season,133115,-1,-1,-1,-1,NaN
43972,770195,MDR vs 3DM,10394,1,130451,132474,130451,d,0,Regular Season,130451,-1,-1,-1,-1,NaN


In [70]:
matches_infos_dataframe = pd.read_pickle("../machine_learning/dataframes/data")

team_games_against_opponent = matches_infos_dataframe.groupby(["team_id", "opponent_id"]).size()

team_games_against_opponent.name = "number_of_games_against_opponent"

matches_infos_dataframe = pd.merge(matches_infos_dataframe, team_games_against_opponent, on=["team_id", "opponent_id"], how='left')

df = matches_infos_dataframe[matches_infos_dataframe["team_id"] == matches_infos_dataframe["winner_id"]]

percentage_of_win_against_opponent = df.groupby(["team_id", "opponent_id"]).size() / team_games_against_opponent * 100
percentage_of_win_against_opponent.name = "percentage_of_win_against_opponent"

matches_infos_dataframe = pd.merge(matches_infos_dataframe, percentage_of_win_against_opponent.to_frame(), on=["team_id", "opponent_id"], how='left')

matches_infos_dataframe

KeyError: 'team_id'

In [1]:
matches_infos_dataframe = pd.read_pickle("../machine_learning/dataframes/data")

matches_infos_dataframe

NameError: name 'pd' is not defined

In [53]:
df = pd.read_pickle("../machine_learning/dataframes/final_dataframe")

a = df[(df["team_id"] == df["winner_id"]) & (df["team_id"] == df["Game 1 winner_id"])].groupby("team_id")["team_id"].count()

b = df.groupby("team_id")["team_id"].count()

c = a / b

df = df.merge(c.to_frame(), left_on="team_id", right_index=True, how="left")

df.rename(columns={"team_id_y": "percentage_of_win_after_won_game_1", "team_id_x": "team_id"}, inplace=True)

df['percentage_of_win_against_opponent'] = df.apply(lambda row: df[(df['team_id'] == row['team_id']) & (df['opponent_id'] == row['opponent_id'])]['winner_id'].value_counts(normalize=True).get(row['team_id'], np.nan) if row['team_id'] != row['opponent_id'] else np.nan, axis=1)

a = df[df["team_id"] == df["winner_id"]].groupby(["number_of_games", "team_id"])["team_id"].count()

b = df.groupby(["number_of_games", "team_id"])["team_id"].count()

c = a / b

df = df.merge(c.to_frame(), left_on=["number_of_games", "team_id"], right_index=True, how="left")

df.rename(columns={"team_id_y": "percentage_of_win_after_n_games", "team_id_x": "team_id"}, inplace=True)

df.to_pickle("../machine_learning/dataframes/final_dataframe")

In [51]:
import pickle
from sklearn.impute import SimpleImputer

df = pd.read_pickle("../machine_learning/dataframes/final_dataframe_modified")

X = df[['match_id', 'winner_id', 'tournament_id', 'number_of_games', 'tournament_tier', 'tournament_has_bracket',
        'percentage_of_win_after_won_game_1', 'percentage_of_win_against_opponent', 'percentage_of_win_after_n_games']]

X = X.apply(pd.to_numeric, errors='coerce')

# Imputer les valeurs manquantes avec la moyenne
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

with open("../machine_learning/data/model.pickle", "rb") as file:
    calibrated_model = pickle.load(file)

for i in range(len(df)):
    prediction = calibrated_model.predict_proba([X[i]])[0][1]
    df.at[i, 'win_probability'] = (1 / prediction)

# Afficher le dataframe mis à jour
df

,match_id,name,tournament_id,number_of_games,winner_id,team_id,opponent_id,tournament_tier,tournament_has_bracket,tournament_name,Game 1 winner_id,Game 2 winner_id,Game 3 winner_id,Game 4 winner_id,Game 5 winner_id,percentage_of_win_after_won_game_1,percentage_of_win_against_opponent,percentage_of_win_after_n_games,win_probability
0,524964,DW vs MMM,2038,1,0,180,2796,None,0,Regular season,0,-1,-1,-1,-1,0.531746,0.882353,0.588785,1.378994
1,524965,MMM vs LGC,2038,1,1,2796,181,None,0,Regular season,1,-1,-1,-1,-1,0.352941,0.538462,0.350877,2.136788
2,524968,BMR vs AV,2038,1,1,1519,179,None,0,Regular season,1,-1,-1,-1,-1,0.714286,1.000000,0.730769,1.108809
3,524970,MMM vs BMR,2038,1,0,2796,1519,None,0,Regular season,0,-1,-1,-1,-1,0.352941,0.333333,0.350877,2.341751
4,524971,ORD vs DW,2038,1,0,180,1521,None,0,Regular season,0,-1,-1,-1,-1,0.531746,0.500000,0.588785,2.224107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43969,770192,BLX vs JL,10394,1,1,132476,132656,d,0,Regular Season,1,-1,-1,-1,-1,0.416667,1.000000,0.454545,1.118344
43970,770193,WYL vs SOL,10394,1,0,133070,130452,d,0,Regular Season,0,-1,-1,-1,-1,0.333333,NaN,0.333333,10.988248
43971,770194,AKR vs GM,10394,1,1,133115,133071,d,0,Regular Season,1,-1,-1,-1,-1,0.666667,1.000000,0.666667,1.093664
43972,770195,MDR vs 3DM,10394,1,0,132474,130451,d,0,Regular Season,0,-1,-1,-1,-1,0.363636,NaN,0.363636,11.207944


In [26]:
df.to_pickle("../machine_learning/dataframes/final_dataframe_odds")

df

,match_id,name,tournament_id,number_of_games,winner_id,team_id,opponent_id,tournament_tier,tournament_has_bracket,tournament_name,Game 1 winner_id,Game 2 winner_id,Game 3 winner_id,Game 4 winner_id,Game 5 winner_id
0,614930,3056,7253,3,1,128268,129970,2,0,37,1,0,1,-1,-1
1,616060,4789,7313,3,0,128217,2882,0,0,29,0,0,-1,-1,-1
2,616061,2702,7313,3,0,2883,130063,0,0,29,0,0,-1,-1,-1
3,616062,1124,7313,3,0,128409,126370,0,0,29,1,0,0,-1,-1
4,616063,3215,7313,3,0,63,126061,0,0,29,0,0,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22929,786070,4578,10658,1,0,130382,132469,3,0,29,0,-1,-1,-1,-1
22930,786566,8040,10724,3,0,131255,132233,2,1,24,0,0,-1,-1,-1
22931,786567,8062,10724,3,0,128538,128581,2,1,24,0,0,-1,-1,-1
22932,786659,6039,10725,3,1,128955,128956,2,0,40,1,1,-1,-1,-1


In [21]:
df = pd.read_pickle("../machine_learning/dataframes/final_dataframe")


columns_to_update = ["Game 1 winner_id", "Game 2 winner_id", "Game 3 winner_id", "Game 4 winner_id", "Game 5 winner_id", "winner_id"]

for column in columns_to_update:
    df[column] = np.where(df[column] == df["team_id"], 1, np.where(df[column] == -1, -1, 0))

df
# for column in df2.columns:
#     df2[column] = df2[column].apply(lambda x: 1 if x == df["team_id"].values else (-1 if x == -1 else 0))
#
# df2

,match_id,name,tournament_id,number_of_games,winner_id,team_id,opponent_id,tournament_tier,tournament_has_bracket,tournament_name,Game 1 winner_id,Game 2 winner_id,Game 3 winner_id,Game 4 winner_id,Game 5 winner_id
0,614930,3056,7253,3,1,128268,129970,2,0,37,1,0,1,-1,-1
1,616060,4789,7313,3,0,128217,2882,0,0,29,0,0,-1,-1,-1
2,616061,2702,7313,3,0,2883,130063,0,0,29,0,0,-1,-1,-1
3,616062,1124,7313,3,0,128409,126370,0,0,29,1,0,0,-1,-1
4,616063,3215,7313,3,0,63,126061,0,0,29,0,0,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22929,786070,4578,10658,1,0,130382,132469,3,0,29,0,-1,-1,-1,-1
22930,786566,8040,10724,3,0,131255,132233,2,1,24,0,0,-1,-1,-1
22931,786567,8062,10724,3,0,128538,128581,2,1,24,0,0,-1,-1,-1
22932,786659,6039,10725,3,1,128955,128956,2,0,40,1,1,-1,-1,-1


In [33]:
from machine_learning.pre_process.data_processing import clean_matches_infos_dataframe

df = pd.read_pickle("../machine_learning/dataframes/final_dataframe_modified")

df

,match_id,name,tournament_id,number_of_games,winner_id,team_id,opponent_id,tournament_tier,tournament_has_bracket,tournament_name,Game 1 winner_id,Game 2 winner_id,Game 3 winner_id,Game 4 winner_id,Game 5 winner_id,percentage_of_win_after_won_game_1,percentage_of_win_against_opponent,percentage_of_win_after_n_games
0,524964,DW vs MMM,2038,1,0,180,2796,None,0,Regular season,0,-1,-1,-1,-1,0.531746,0.882353,0.588785
1,524965,MMM vs LGC,2038,1,1,2796,181,None,0,Regular season,1,-1,-1,-1,-1,0.352941,0.538462,0.350877
2,524968,BMR vs AV,2038,1,1,1519,179,None,0,Regular season,1,-1,-1,-1,-1,0.714286,1.000000,0.730769
3,524970,MMM vs BMR,2038,1,0,2796,1519,None,0,Regular season,0,-1,-1,-1,-1,0.352941,0.333333,0.350877
4,524971,ORD vs DW,2038,1,0,180,1521,None,0,Regular season,0,-1,-1,-1,-1,0.531746,0.500000,0.588785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43969,770192,BLX vs JL,10394,1,1,132476,132656,d,0,Regular Season,1,-1,-1,-1,-1,0.416667,1.000000,0.454545
43970,770193,WYL vs SOL,10394,1,0,133070,130452,d,0,Regular Season,0,-1,-1,-1,-1,0.333333,NaN,0.333333
43971,770194,AKR vs GM,10394,1,1,133115,133071,d,0,Regular Season,1,-1,-1,-1,-1,0.666667,1.000000,0.666667
43972,770195,MDR vs 3DM,10394,1,0,132474,130451,d,0,Regular Season,0,-1,-1,-1,-1,0.363636,NaN,0.363636


In [4]:
df = pd.read_pickle("../machine_learning/dataframes/randomforest")

df

,match_id,name,tournament_id,number_of_games,winner_id,team_id,opponent_id,tournament_tier,tournament_has_bracket,tournament_name,Game 1 winner_id,Game 2 winner_id,Game 3 winner_id,Game 4 winner_id,Game 5 winner_id,percentage_of_win_after_won_game_1,percentage_of_win_against_opponent,percentage_of_win_after_n_games
0,614930,3059,7253,3,1,128268,129970,2,0,38,1,0,1,-1,-1,0.589286,1.000000,1.000000
1,616060,4810,7313,3,0,128217,2882,0,0,30,0,0,-1,-1,-1,0.200000,NaN,0.200000
2,616061,2705,7313,3,0,2883,130063,0,0,30,0,0,-1,-1,-1,0.242424,0.250000,0.266667
3,616062,1124,7313,3,0,128409,126370,0,0,30,1,0,0,-1,-1,0.515152,0.333333,0.550000
4,616063,3219,7313,3,0,63,126061,0,0,30,0,0,-1,-1,-1,0.500000,NaN,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23025,786659,6060,10725,3,1,128955,128956,2,0,41,1,1,-1,-1,-1,1.000000,1.000000,1.000000
23026,787703,1614,10734,3,1,132428,130400,3,0,25,1,1,-1,-1,-1,0.625000,0.600000,0.666667
23027,788128,6201,10725,3,0,128982,128955,2,0,41,0,0,-1,-1,-1,0.333333,NaN,0.285714
23028,788438,3819,10584,5,0,133039,130375,3,1,24,1,0,0,0,-1,0.272727,NaN,NaN


In [34]:
df = pd.read_pickle("../machine_learning/data/df/matches_df")

df

,match_id,name,tournament_id,number_of_games,winner_id,team_id,opponent_id,tournament_tier,tournament_has_bracket,tournament_name,Game 1 winner_id,Game 2 winner_id,Game 3 winner_id,Game 4 winner_id,Game 5 winner_id
0,541048,BRB vs BBQ,2088,3,57988,57988,403,None,0,Regular season,-1,-1,-1,-1,-1
1,543705,DG vs SE,2288,5,125925,124409,125925,b,0,Promotion,-1,-1,-1,-1,-1
2,544125,RWS vs TSG,2326,5,125993,125991,125993,None,1,Playoffs,-1,-1,-1,-1,-1
3,552052,Elimination Match 4: HLE vs ESC,3451,3,-1,2883,126708,None,1,Elimination and Qualifying Rounds,-1,-1,-1,-1,-1
4,555678,M19 vs VEG,3733,1,663,663,664,None,0,Group D,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,789032,Upper bracket semifinal 2: RAAD vs TEC,10756,3,-1,132738,129614,d,1,Playoffs,-1,-1,-1,-1,-1
2102,789033,Lower bracket semifinal: TBD vs TBD,10756,3,-1,-1,-1,d,1,Playoffs,-1,-1,-1,-1,-1
2103,789034,Upper bracket final: TBD vs TBD,10756,3,-1,-1,-1,d,1,Playoffs,-1,-1,-1,-1,-1
2104,789035,Lower bracket final: TBD vs TBD,10756,3,-1,-1,-1,d,1,Playoffs,-1,-1,-1,-1,-1


In [39]:
df2 = pd.read_pickle("../machine_learning/data/df/old_matches_df")

for index, value in df.iterrows():
    serie = df2[df2["team_id"] == value["team_id"]]["percentage_of_win_after_won_game_1"]
    value = serie.values[0] if not serie.empty else np.nan

    df.loc[index, "percentage_of_win_after_won_game_1"] = value

    serie = df2[df2["team_id"] == value["team_id"]]["percentage_of_win_against_opponent"]
    value = serie.values[0] if not serie.empty else np.nan

    df.loc[index, "percentage_of_win_against_opponent"] = value

    serie = df2[df2["team_id"] == value["team_id"]]["percentage_of_win_after_n_games"]
    value = serie.values[0] if not serie.empty else np.nan

    df.loc[index, "percentage_of_win_after_n_games"] = value

df

,match_id,name,tournament_id,number_of_games,winner_id,team_id,opponent_id,tournament_tier,tournament_has_bracket,tournament_name,Game 1 winner_id,Game 2 winner_id,Game 3 winner_id,Game 4 winner_id,Game 5 winner_id,percentage_of_win_after_won_game_1,percentage_of_win_against_opponent,percentage_of_win_after_n_games
0,541048,BRB vs BBQ,2088,3,57988,57988,403,None,0,Regular season,-1,-1,-1,-1,-1,NaN,NaN,NaN
1,543705,DG vs SE,2288,5,125925,124409,125925,b,0,Promotion,-1,-1,-1,-1,-1,NaN,NaN,NaN
2,544125,RWS vs TSG,2326,5,125993,125991,125993,None,1,Playoffs,-1,-1,-1,-1,-1,NaN,NaN,NaN
3,552052,Elimination Match 4: HLE vs ESC,3451,3,-1,2883,126708,None,1,Elimination and Qualifying Rounds,-1,-1,-1,-1,-1,0.242424,0.25,0.266667
4,555678,M19 vs VEG,3733,1,663,663,664,None,0,Group D,-1,-1,-1,-1,-1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,789032,Upper bracket semifinal 2: RAAD vs TEC,10756,3,-1,132738,129614,d,1,Playoffs,-1,-1,-1,-1,-1,0.677419,0.40,0.764706
2102,789033,Lower bracket semifinal: TBD vs TBD,10756,3,-1,-1,-1,d,1,Playoffs,-1,-1,-1,-1,-1,NaN,NaN,NaN
2103,789034,Upper bracket final: TBD vs TBD,10756,3,-1,-1,-1,d,1,Playoffs,-1,-1,-1,-1,-1,NaN,NaN,NaN
2104,789035,Lower bracket final: TBD vs TBD,10756,3,-1,-1,-1,d,1,Playoffs,-1,-1,-1,-1,-1,NaN,NaN,NaN


In [17]:
import mysql.connector

df = pd.read_pickle("../machine_learning/data/matches_df")

conn = mysql.connector.connect(
    host='34.155.63.44',
    user='admin',
    password='azertyuiop',
    database='main'
)

cursor = conn.cursor()

df["winner_prediction"] = df["winner_prediction"].round(2)

for index, row in df.iterrows():
    team_id = row['team_id']
    opponent_id = row['opponent_id']
    match_id = row['match_id']
    winner_prediction = row['winner_prediction']

    update_query = f"""
    UPDATE matchs
    SET home_odd = CASE
        WHEN {team_id} = home_id THEN {winner_prediction}
        ELSE home_odd
        END,
        away_odd = CASE
        WHEN {team_id} = away_id THEN {winner_prediction}
        ELSE away_odd
        END
    WHERE id = {match_id};
    """
    cursor.execute(update_query)

conn.commit()
conn.close()

In [54]:
sc = StandardScaler()

dataframe = pd.read_pickle("../machine_learning/data/df/old_matches_df")

x_dataframe = dataframe[['number_of_games', 'team_id', 'opponent_id', 'tournament_tier', 'percentage_of_win_after_won_game_1', 'percentage_of_win_against_opponent', 'percentage_of_win_after_n_games']]
y_dataframe = dataframe["winner_id"]

# y_dataframe = dataframe[["winner_id", 'Game 1 winner_id', 'Game 2 winner_id', 'Game 3 winner_id', 'Game 4 winner_id', 'Game 5 winner_id']]

label_encoder = LabelEncoder()
x_dataframe.loc[:, 'tournament_tier'] = label_encoder.fit_transform(x_dataframe['tournament_tier'])

simple_imputer = SimpleImputer(strategy='mean')
x_dataframe = simple_imputer.fit_transform(x_dataframe)

x_train, x_test, y_train, y_test = train_test_split(x_dataframe, y_dataframe, test_size=0.2)

x_train_scaled = sc.fit_transform(x_train)
x_test_scaled = sc.transform(x_test)

# print(pd.DataFrame(x_train_scaled))

best_params = {'C': 0.01, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'newton-cg'}

model = LogisticRegression(n_jobs=-1)

model.fit(x_train_scaled, y_train)

calibrated_model = CalibratedClassifierCV(model, method='sigmoid', cv=5, n_jobs=-1)

calibrated_model.fit(x_train_scaled, y_train)

pred = calibrated_model.predict_proba(x_test_scaled)

from sklearn.model_selection import cross_val_score

scores = cross_val_score(calibrated_model, x_train_scaled, y_train, cv=5, n_jobs=-1)

print("Scores de validation croisée :", scores)

print("Score moyen de validation croisée :", scores.mean())

pred[:, 1]

# Il faudrait peut-être essayer de chercher les hyper-paramètres avec les données "scaled", ça pourrait peut-être augmenter le score

C:\Users\talha\AppData\Local\Temp\ipykernel_21920\1531826481.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_dataframe.loc[:, 'tournament_tier'] = label_encoder.fit_transform(x_dataframe['tournament_tier'])


Scores de validation croisée : [0.76443481 0.74762808 0.76524803 0.75684467 0.7516269 ]
Score moyen de validation croisée : 0.7571564991541365


array([0.28924053, 0.68508887, 0.87136448, ..., 0.27357237, 0.37420465,
       0.65971338])

In [69]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

sc = StandardScaler()

dataframe = pd.read_pickle("../machine_learning/data/df/old_matches_df")

x_dataframe = dataframe[['number_of_games', 'team_id', 'opponent_id', 'tournament_tier', 'percentage_of_win_after_won_game_1', 'percentage_of_win_against_opponent', 'percentage_of_win_after_n_games']]
y_dataframe = dataframe["winner_id"]

label_encoder = LabelEncoder()
x_dataframe.loc[:, 'tournament_tier'] = label_encoder.fit_transform(x_dataframe['tournament_tier'])

simple_imputer = SimpleImputer(strategy='constant', fill_value=0)
x_dataframe = simple_imputer.fit_transform(x_dataframe)

x_train, x_test, y_train, y_test = train_test_split(x_dataframe, y_dataframe, test_size=0.2)

x_train_scaled = sc.fit_transform(x_train)
x_test_scaled = sc.transform(x_test)

model = GradientBoostingClassifier(**{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300})

model.fit(x_train_scaled, y_train)

pred = model.predict_proba(x_test_scaled)

scores = cross_val_score(model, x_train_scaled, y_train, cv=5, n_jobs=-1)

print("Scores de validation croisée :", scores)
print("Score moyen de validation croisée :", scores.mean())

pred[:, 1]


C:\Users\talha\AppData\Local\Temp\ipykernel_21920\2387235537.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_dataframe.loc[:, 'tournament_tier'] = label_encoder.fit_transform(x_dataframe['tournament_tier'])


AttributeError: 'SVR' object has no attribute 'predict_proba'

In [60]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 4, 5]
}

model = GradientBoostingClassifier()

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(model, param_grid, cv=stratified_kfold, n_jobs=-1)

grid_search.fit(x_train_scaled, y_train)

print("Meilleurs hyperparamètres :", grid_search.best_params_)

best_model = grid_search.best_estimator_

Meilleurs hyperparamètres : {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300}


In [70]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

sc = StandardScaler()

dataframe = pd.read_pickle("../machine_learning/data/df/old_matches_df")

x_dataframe = dataframe[['number_of_games', 'team_id', 'opponent_id', 'tournament_tier', 'percentage_of_win_after_won_game_1', 'percentage_of_win_against_opponent', 'percentage_of_win_after_n_games']]
y_dataframe = dataframe["winner_id"]

label_encoder = LabelEncoder()
x_dataframe.loc[:, 'tournament_tier'] = label_encoder.fit_transform(x_dataframe['tournament_tier'])

simple_imputer = SimpleImputer(strategy='constant', fill_value=0)
x_dataframe = simple_imputer.fit_transform(x_dataframe)

x_train, x_test, y_train, y_test = train_test_split(x_dataframe, y_dataframe, test_size=0.2)

x_train_scaled = sc.fit_transform(x_train)
x_test_scaled = sc.transform(x_test)

model = SVC(probability=True)

model.fit(x_train_scaled, y_train)

pred = model.predict_proba(x_test_scaled)

scores = cross_val_score(model, x_train_scaled, y_train, cv=5, n_jobs=-1)

print("Scores de validation croisée :", scores)
print("Score moyen de validation croisée :", scores.mean())

pred[:, 1]


C:\Users\talha\AppData\Local\Temp\ipykernel_21920\1544044274.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_dataframe.loc[:, 'tournament_tier'] = label_encoder.fit_transform(x_dataframe['tournament_tier'])


Scores de validation croisée : [0.81621036 0.83599892 0.83057739 0.83464353 0.83541215]
Score moyen de validation croisée : 0.8305684690782058


array([0.76546111, 0.74725188, 0.60104639, ..., 0.10807571, 0.15706913,
       0.1290515 ])

In [88]:

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


sc = StandardScaler()

dataframe = pd.read_pickle("../machine_learning/data/df/old_matches_df")

x_dataframe = dataframe[['number_of_games', 'team_id', 'opponent_id', 'tournament_tier', 'percentage_of_win_after_won_game_1', 'percentage_of_win_against_opponent', 'percentage_of_win_after_n_games']]
y_dataframe = dataframe["winner_id"]

label_encoder = LabelEncoder()
x_dataframe.loc[:, 'tournament_tier'] = label_encoder.fit_transform(x_dataframe['tournament_tier'])

simple_imputer = SimpleImputer(strategy='constant', fill_value=0)
x_dataframe = simple_imputer.fit_transform(x_dataframe)

x_train, x_test, y_train, y_test = train_test_split(x_dataframe, y_dataframe, test_size=0.2)

x_train_scaled = sc.fit_transform(x_train)
x_test_scaled = sc.transform(x_test)

model = KNeighborsClassifier()

model.fit(x_train_scaled, y_train)

pred = model.predict_proba(x_test_scaled)

scores = cross_val_score(model, x_train_scaled, y_train, cv=5, n_jobs=-1)

print("Scores de validation croisée :", scores)
print("Score moyen de validation croisée :", scores.mean())

pred[:, 1]

C:\Users\talha\AppData\Local\Temp\ipykernel_21920\3228496643.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_dataframe.loc[:, 'tournament_tier'] = label_encoder.fit_transform(x_dataframe['tournament_tier'])


Scores de validation croisée : [0.80346978 0.80184332 0.80455408 0.80645161 0.80775488]
Score moyen de validation croisée : 0.8048147332545781


array([0.92645093, 0.92645093, 0.07550962, ..., 0.07550962, 0.37994549,
       0.92645093])